# 12. Higher-Order Functions

Scala mixes object orientation with functional features. In a functional programming language, functions are fi rst-class citizens that can be passed around and
manipulated just like any other data types. This is very useful whenever you
want to pass some action detail to an algorithm. In a functional language, you
just wrap that detail into a function that you pass as a parameter. In this chapter,
you will see how to be productive with functions that use or return functions.


## 12.1 Functions as Values

In Scala, a function is a fi rst-class citizen, just like a number. You can store a
function in a variable:


In [1]:
import scala.math._
val num = 3.14
val fun = ceil _

import scala.math._

num: Double = 3.14
fun: Double => Double = <function1>

This code sets num to 3.14 and fun to the ceil function.
The _ behind the ceil function indicates that you really meant the function, and
you didn’ t just forget to supply the arguments.
When you try this code in the REPL, the type of num is, not surprisingly, Double.
The type of fun is reported as (Double) => Double— that is, a function receiving and
returning a Double

#### NOTE: 
Technically, the _ turns the ceil method into a function. In Scala, you
cannot manipulate methods, only functions.The type of the function is (Double)
=> Double, with an arrow. In contrast, the type of the ceil method is
(Double)Double, without an arrow. There is no way for you to work with such a
type, but you will fi nd it in compiler and REPL messages.
The _ suffi x is not necessary when you use a method name in a context where
a function is expected. For example, the following is legal:
val f: (Double) => Double = ceil // No underscore needed

#### NOTE: 
The ceil method is a method of the scala.math package object. If you
have a method from a class, the syntax for turning it into a function is slightly
different:
val f = (_: String).charAt(_: Int)
 // A function (String, Int) => Char
Alternatively, you can specify the type of the function instead of the parameter
types:
val f: (String, Int) => Char = _.charAt(_)

What can you do with a function? Two things:

• Call it.

• Pass it around, by storing it in a variable or giving it to a function as a

parameter.
Here is how to call the function stored in fun:

`fun(num) // 4.0`

As you can see, the normal function call syntax is used. The only difference is
that fun is a variable containing a function, not a fi xed function.


Here is how you can give fun to another function:

In [2]:
Array(3.14, 1.42, 2.0).map(fun) // Array(4.0, 2.0, 2.0)

res1: Array[Double] = Array(4.0, 2.0, 2.0)

The map method accepts a function, applies it to all values in an array, and returns
an array with the function values. In this chapter, you will see many other
methods that accept functions as parameters.

## 12.2 Anonymous Functions

In Scala, you don’ t have to give a name to each function, just like you don’ t have
to give a name to each number. Here is an anonymous function:

In [1]:
(x: Double) => 3 * x

res0: Double => Double = <function1>

This function multiplies its argument by 3.

Of course, you can store this function in a variable:


In [9]:
val triple = (x: Double) => 3 * x

triple: Double => Double = <function1>
res8_1: Class[?0] = class $sess.cmd8Wrapper$Helper$$anonfun$1

That’ s just as if you had used a def:

In [10]:
def triple(x: Double) = 3 * x

defined function triple

But you don’ t have to name the function. You can just pass it to another function:

In [4]:
Array(3.14, 1.42, 2.0).map((x: Double) => 3 * x)
 // Array(9.42, 4.26, 6.0)

res3: Array[Double] = Array(9.42, 4.26, 6.0)

Here, we tell the map method: “ Multiply each element by 3.”

#### NOTE: 
If you prefer, you can enclose the function argument in braces instead
of parentheses, for example:
Array(3.14, 1.42, 2.0).map{ (x: Double) => 3 * x }
This is more common when a method is used in infi x notation (without the dot).
Array(3.14, 1.42, 2.0) map { (x: Double) => 3 * x }

#### NOTE: 
Anything defi ned with def (in the REPL or a class or object) is a
method, not a function:
scala> def triple(x: Double) = 3 * x
triple: (x: Double)Double
Note the method type (x: Double)Double. In contrast, a function defi nition has
a function type:
scala> val triple = (x: Double) => 3 * x
triple: Double => Double

In [12]:
object test{
    def mult(a: Int, b: Int) :Int = a*b
    val mult2 = (a:Int, b:Int) => a*b
}

defined object test

below commands are equal

In [17]:
test.mult _
test.mult2

res16_0: (Int, Int) => Int = <function2>
res16_1: (Int, Int) => Int = <function2>

## 12.3 Functions with Function Parameters

In this section, you will see how to implement a function that takes another
function as a parameter. Here is an example:

In [18]:
def valueAtOneQuarter(f: (Double) => Double) = f(0.25)

defined function valueAtOneQuarter

Note that the parameter can be any function receiving and returning a Double. The
valueAtOneQuarter function computes the value of that function at 0.25.

In [21]:
import scala.math._

import scala.math._

In [22]:
valueAtOneQuarter(ceil _) // 1.0
valueAtOneQuarter(sqrt _) // 0.5 (because 0.5 × 0.5 = 0.25)

res21_0: Double = 1.0
res21_1: Double = 0.5

What is the type of valueAtOneQuarter? It is a function with one parameter, so its
type is written as

`( parameterType) => resultType`

The resultType is clearly Double, and the parameterType is already given in the function
header as (Double) => Double. Therefore, the type of valueAtOneQuarter is

`((Double) => Double) => Double
`

Since valueAtOneQuarter is a function that receives a function, it is called a
higher-order function.
A higher-order function can also produce a function. Here is a simple example:

In [23]:
def mulBy(factor : Double) = (x : Double) => factor * x

defined function mulBy

For example, mulBy(3) returns the function (x : Double) => 3 * x which you have
seen in the preceding section. The power of mulBy is that it can deliver functions
that multiply by any amount

In [24]:
val quintuple = mulBy(5)
quintuple(20) // 100

quintuple: Double => Double = <function1>
res23_1: Double = 100.0

The mulBy function has a parameter of type Double, and it returns a function of type
(Double) => Double. Therefore, its type is

`(Double) => ((Double) => Double)
`

When you pass an anonymous function to another function or method, Scala
helps you out by deducing types when possible. For example, you don’ t have to
write

In [26]:
valueAtOneQuarter((x: Double) => 3 * x) // 0.75

res25: Double = 0.75

Since the valueAtOneQuarter method knows that you will pass in a (Double) => Double
function, you can just write

In [27]:
valueAtOneQuarter((x) => 3 * x)

res26: Double = 0.75

As a special bonus, for a function that has just one parameter, you can omit the
() around the parameter:


In [28]:
valueAtOneQuarter(x => 3 * x)

res27: Double = 0.75

It gets better. If a parameter occurs only once on the right-hand side of the =>,
you can replace it with an underscore:


In [29]:
valueAtOneQuarter(3 * _)

res28: Double = 0.75

This is the ultimate in comfort, and it is also pretty easy to read: a function that
multiplies something by 3.
Keep in mind that these shortcuts only work when the parameter types are
known.


In [32]:
val fun = 3 * _ // Error: Can’ t infer types
val fun = 3 * (_: Double) // OK
val fun: (Double) => Double = 3 * _ // OK because we specifi ed the type for fun

cmd32.sc:2: fun is already defined as value fun
val fun = 3 * (_: Double) // OK
            ^cmd32.sc:3: fun is already defined as value fun
val fun: (Double) => Double = 3 * _ // OK because we specifi ed the type for fun
                                ^cmd32.sc:1: missing parameter type for expanded function ((x$1) => 3.*(x$1))
val fun = 3 * _ // Error: Can’ t infer types
              ^

: 

Of course, the last defi nition is contrived. But it shows what happens when a
function is passed to a parameter (which has just such a type).

#### NOTE: 

Specifying the type of _ is useful for turning methods into functions.
For example, (_: String).length is a function String => Int, and
(_: String).substring(_:Int, _: Int) is a function (String, Int, Int) => String.

## 12.5 Useful Higher-Order Functions

A good way of becoming comfortable with higher-order functions is to practice
with some common (and obviously useful) methods in the Scala collections library
that take function parameters.
You have seen map, which applies a function to all elements of a collection and
returns the result. Here is a quick way of producing a collection containing 0.1,
0.2, . . . , 0.9:

In [35]:
(1 to 9).map(0.1 * _)

res34: collection.immutable.IndexedSeq[Double] = Vector(
  0.1,
  0.2,
  0.30000000000000004,
  0.4,
  0.5,
  0.6000000000000001,
  0.7000000000000001,
  0.8,
  0.9
)

Try this to print a triangle:


In [36]:
(1 to 9).map("*" * _).foreach(println _)

*
**
***
****
*****
******
*******
********
*********


Here, we also use foreach, which is like map except that its function doesn’ t return
a value. The foreach method simply applies the function to each argument.
The filter method yields all elements that match a particular condition. For
example, here’ s how to get only the even numbers in a sequence:

In [37]:
(1 to 9).filter(_ % 2 == 0) // 2, 4, 6, 8

res36: collection.immutable.IndexedSeq[Int] = Vector(2, 4, 6, 8)

Of course, that’ s not the most effi cient way of getting this result .
The reduceLeft method takes a binary function— that is, a function with two param-
eters— and applies it to all elements of a sequence, going from left to right. For
example,

In [38]:
(1 to 9).reduceLeft(_ * _)

res37: Int = 362880

is

`1 * 2 * 3 * 4 * 5 * 6 * 7 * 8 * 9
`
or, strictly speaking,

(...((1 * 2) * 3) * ... * 9)


Note the compact form of the multiplication function: _ * _. Each underscore
denotes a separate parameter.
You also need a binary function for sorting. For example,

In [40]:
"Mary had a little lamb".split(" ").sortWith(_.length < _.length)

res39: Array[String] = Array("a", "had", "Mary", "lamb", "little")

## 12.6 Closures
(Scala Tutorial)

A closure is a function, whose return value depends on the value of one or more variables declared
outside this function. Consider the following piece of code with anonymous function:


In [43]:
val multiplier = (i:Int) => i * 10

multiplier: Int => Int = <function1>

Here, the only variable used in the function body, i * 0, is i, which is defined as a parameter to the function. Now, let
us take another piece of code:

In [43]:
val multiplier = (i:Int) => i * factor

cmd43.sc:1: not found: value factor
val multiplier = (i:Int) => i * factor
                                ^

: 

There are two free variables in multiplier: i and factor. One of them, i, is a formal parameter to the function. Hence,
it is bound to a new value each time multiplier is called. However, factor is not a formal parameter, then what is
this? Let us add one more line of code:

In [44]:
var factor = 3
val multiplier = (i:Int) => i * factor

factor: Int = 3
multiplier: Int => Int = <function1>

Now, factor has a reference to a variable outside the function but in the enclosing scope. Let us try the following
example:

In [46]:
object Test {
var factor = 3
val multiplier = (i:Int) => i * factor
}

defined object Test

In [47]:
Test.multiplier(1) 
Test.multiplier(2) 

res46_0: Int = 3
res46_1: Int = 6

Above function references factor and reads its current value each time. If a function has no external references,
then it is trivially closed over itself. No external context is required.

## 12.8 Currying

Currying (named after logician Haskell Brooks Curry) is the process of turning
a function that takes two arguments into a function that takes one argument.
That function returns a function that consumes the second argument.
Huh? Let’ s look at an example. This function takes two arguments:

In [48]:
val mul = (x: Int, y: Int) => x * y


mul: (Int, Int) => Int = <function2>

This function takes one argument, yielding a function that takes one argument:

In [49]:
val mulOneAtATime = (x: Int) => ((y: Int) => x * y)

mulOneAtATime: Int => Int => Int = <function1>

To multiply two numbers, you call

In [50]:
mulOneAtATime(6)(7)

res49: Int = 42

Strictly speaking, the result of mulOneAtATime(6) is the function (y: Int) => 6 * y. That
function is applied to 7 , yielding 42.
When you use def, there is a shortcut for defi ning such curried methods in Scala:

In [51]:
def mulOneAtATime(x: Int)(y: Int) = x * y

defined function mulOneAtATime

#### Note:
NOTE: Recall that anything defi ned with def (in the REPL or a class or object)
is a method, not a function. When defi ning curried methods with def, you can
use multiple parentheses:

`scala> def mulOneAtATime(x: Int)(y: Int) = x * y

mulOneAtATime: (x: Int)(y: Int)Int
`

Note the method type (x: Int)(y: Int)Int. In contrast, when you defi ne a
function, you must use multiple arrows, not multiple parentheses:

`
scala> val mulOneAtATime = (x: Int) => (y: Int) => x * y

mulOneAtATime: Int => (Int => Int)
`

As you can see, multiple parameters are just a frill, not an essential feature of
a programming language. That’ s an amusing theoretical insight, but it has one
practical use in Scala. Sometimes, you can use currying for a method parameter
so that the type inferencer has more information.
Here is a typical example. The corresponds method can compare whether two
sequences are the same under some comparison criterion. For example,

In [52]:
val a = Array("Hello", "World")
val b = Array("hello", "world")
a.corresponds(b)(_.equalsIgnoreCase(_))

a: Array[String] = Array("Hello", "World")
b: Array[String] = Array("hello", "world")
res51_2: Boolean = true

Note that the function _.equalsIgnoreCase(_) is passed as a curried parameter, in a
separate set of (...). When you look into the Scaladoc, you will see that corresponds
is declared as

`def corresponds[B](that: Seq[B])(p: (A, B) => Boolean): Boolean 
`

The that sequence and the predicate function p are separate curried parame-
ters. The type inferencer can fi gure out what B is from the type of that, and then
it can use that information when analyzing the function that is passed for p .
In our example, that is a String sequence. Therefore, the predicate is expected to
have type (String, String) => Boolean. With that information, the compiler can accept _.equalsIgnoreCase(_) as a shortcut for (a: String, b: String) => a.equalsIgnoreCase(b).


In [58]:
val mult = (x:Int, y:Int) => (def p = ((z:Int) => z)) 

: 

## 12.9 Control Abstractions


In [60]:
def runInThread(block: => Unit) { 
 new Thread {
 override def run() { block }
 }.start()
}


defined function runInThread

Then the call becomes simply

In [65]:
runInThread { println("Hi"); Thread.sleep(1); println("Bye") }

Hi


## 12.11 Partially Applied Functions

When you invoke a function, you're said to be applying the function to the arguments. If you pass all the expected
arguments, you have fully applied it. If you send only a few arguments, then you get back a partially applied
function. This gives you the convenience of binding some arguments and leaving the rest to be filled in later.
Following is a simple example to show the concept:

In [75]:
import java.util.Date
val date = new Date
log(date, "message1" )
log(date, "message2" )
log(date, "message3" )

def log(date: Date, message: String){
println(date + "----" + message)
}

Sat Sep 01 18:09:23 IRDT 2018----message1
Sat Sep 01 18:09:23 IRDT 2018----message2
Sat Sep 01 18:09:23 IRDT 2018----message3


import java.util.Date

date: java.util.Date = Sat Sep 01 18:09:23 IRDT 2018
defined function log

Here, the log( ) method takes two parameters: date and message. We want to invoke the method multiple times,
with the same value for date but different values for message. We can eliminate the noise of passing the date to
each call by partially applying that argument to the log( ) method. To do so, we first bind a value to the date
parameter and leave the second parameter unbound by putting an underscore at its place. The result is a partially
applied function that we've stored in a variable. We can now invoke this new method with only the unbound
argument message as follows:

In [76]:
import java.util.Date
val logWithDateBound = log(new Date, _ : String)
logWithDateBound("message1" )
logWithDateBound("message2" )
logWithDateBound("message3" )

def log(date: Date, message: String){
println(date + "----" + message)
}


Sat Sep 01 18:10:22 IRDT 2018----message1
Sat Sep 01 18:10:22 IRDT 2018----message2
Sat Sep 01 18:10:22 IRDT 2018----message3


import java.util.Date

logWithDateBound: String => Unit = <function1>
defined function log